# Diversity Evolutionary Algorithm

In [ ]:

import matplotlib.pyplot as plt

# from examples.z_ec_course.A3_template import NUM_OF_MODULES
import numpy as np

# from ariel_experiments.characterize.individual import analyze_neighbourhood
# from ariel_experiments.characterize.population import (
# get_full_analyzed_population,
# matrix_derive_neighbourhood,
# matrix_derive_neighbourhood_cross_pop,
# )
import torch
import umap
from rich.console import Console
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from ariel.body_phenotypes.robogen_lite.decoders.hi_prob_decoding import (
    HighProbabilityDecoder,
)
from ariel.ec.a004 import (
    EA,
    EASettings,
    EAStep,
    Individual,
    Population,
)
from ariel.ec.genotypes.nde.nde import NeuralDevelopmentalEncoding
from ariel_experiments.characterize.canonical.core.toolkit import (
    CanonicalToolKit as ctk,
)

console = Console()
SEED = 42

# Seed everything for determinism
np.random.seed(SEED)
RNG = np.random.default_rng(SEED)

# Seed PyTorch for deterministic behavior
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True, warn_only=True)

# Settings

In [ ]:
# Global var
global_fitness_history = []
global_best_fitness_history = []
diversity_archive = []  # NEW: Archive of past tree hashes for novelty search

# EA settings
EA_CONFIG = EASettings(
    is_maximisation=True,
    num_of_generations=50,
    target_population_size=100,
)

# evaluation settings
SIM_CONFIG = ctk.SimilarityConfig()
SIM_CONFIG.max_tree_radius = 10
SIM_CONFIG.radius_strategy = ctk.CollectionStrategy.SUBTREES


NUM_OF_MODULES = 20
GENOTYPE_SIZE = 64
SCALE = 8192  # ADDED: Following A3_template pattern

MUTATION_RATE = 0.05  # FIXED: 5% mutation rate (was 0)

# Selection pressure settings
PARENT_TOURNAMENT_SIZE = 4
SURVIVOR_TOURNAMENT_SIZE = 4

# Archive settings
MAX_ARCHIVE_SIZE = 500_000 

GLOBAL_N_NEIGHBORS = EA_CONFIG.target_population_size - 1

K_NEIGHBORS = 5

# ADDED: Global NDE for deterministic genotype-to-phenotype mapping (following A3_template pattern)
GLOBAL_NDE = NeuralDevelopmentalEncoding(number_of_modules=NUM_OF_MODULES)

# Helper functions for EA

In [ ]:
##################################
def record_mean_fitness(population: Population) -> Population:
    # Only calculate mean for alive individuals
    mean_fitness = np.mean([ind.fitness for ind in population if ind.alive])
    global_fitness_history.append(mean_fitness)
    return population


def record_best_fitness(population: Population) -> Population:
    if EA_CONFIG.is_maximisation:
        best_fitness = np.max([ind.fitness for ind in population if ind.alive])
    else:
        best_fitness = np.min([ind.fitness for ind in population if ind.alive])

    global_best_fitness_history.append(best_fitness)
    return population


def add_survivors_to_archive(population: Population) -> Population:
    """
    Add only surviving individuals to the diversity archive.
    This should be called AFTER survivor selection.
    """
    global diversity_archive

    # Get only alive individuals
    survivors = [ind for ind in population if ind.alive]

    # Collect their subtrees
    survivor_subtrees = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_nde_genotype(ind.genotype_),
            config=SIM_CONFIG,
        ) for ind in survivors
    ]

    # Add to archive
    diversity_archive.extend(survivor_subtrees)

    # Limit archive size (keep most recent)
    if len(diversity_archive) > MAX_ARCHIVE_SIZE:
        console.print(f"Archive max size reached at generation {len(global_fitness_history)}")
        diversity_archive = diversity_archive[-MAX_ARCHIVE_SIZE:]

    console.print(f"Added {len(survivor_subtrees)} survivors to archive. Total archive size: {len(diversity_archive)}")

    return population

# EA functions

In [ ]:
def decode_genotype_to_string(genotype: list) -> str:
    """Helper: decode genotype to phenotype string using GLOBAL_NDE."""
    matrixes = GLOBAL_NDE.forward(np.array(genotype))
    hpd = HighProbabilityDecoder(num_modules=NUM_OF_MODULES)
    graph = hpd.probability_matrices_to_graph(matrixes[0], matrixes[1], matrixes[2])
    tree = ctk.from_graph(graph)
    return ctk.to_string(tree)


def float_creep(
    individual: list[list[float]] | list[list[list[float]]],
    mutation_probability: float,
    mutation_step_size: float = 100.0,  # Small step relative to SCALE
) -> list[list[float]]:
    """
    Creep mutation: adds small random perturbations to genes.
    
    Args:
        individual: Genotype to mutate
        mutation_probability: Probability each gene mutates (0-1)
        mutation_step_size: Maximum step size for mutation (default 100)
    """
    ind_arr = np.array(individual)
    shape = ind_arr.shape

    # Generate SMALL mutation values (not full range!)
    mutator = RNG.uniform(-mutation_step_size, mutation_step_size, size=shape)

    # Determine which positions to mutate
    do_mask = RNG.choice(
        [1, 0],
        size=shape,
        p=[mutation_probability, 1 - mutation_probability],
    )
    
    mutation_mask = mutator * do_mask
    new_genotype = ind_arr + mutation_mask
    return new_genotype.tolist()


def make_random_robot(genotype_size: int = GENOTYPE_SIZE) -> Individual:
    """
    Produces a robot with only its genotype.
    Following A3_template.py pattern: uses uniform distribution from -SCALE to +SCALE.
    """
    ind = Individual()
    ind.genotype = [
        RNG.uniform(-SCALE, SCALE, genotype_size).tolist(),
        RNG.uniform(-SCALE, SCALE, genotype_size).tolist(),
        RNG.uniform(-SCALE, SCALE, genotype_size).tolist(),
    ]
    ind.fitness = 0.0
    ind.requires_eval = True
    ind.tags['ctk_string'] = decode_genotype_to_string(ind.genotype)
    return ind


def parent_selection_tournament(population: Population) -> Population:
    """
    K-tournament parent selection with configurable tournament size.
    Higher tournament size = higher selection pressure.
    """
    tournament_size = PARENT_TOURNAMENT_SIZE

    # Run tournaments to select parents
    num_parents = len(population) // 2  # Select ~50% as parents
    selected_parents = []

    for _ in range(num_parents):
        # Randomly select tournament_size individuals
        competitors = RNG.choice(population, size=min(tournament_size, len(population)), replace=False)

        # Find the best in the tournament
        if EA_CONFIG.is_maximisation:
            winner = max(competitors, key=lambda ind: ind.fitness)
        else:
            winner = min(competitors, key=lambda ind: ind.fitness)

        selected_parents.append(winner)

    # Clear all ps tags first
    for ind in population:
        ind.tags["ps"] = False

    # Tag selected parents
    for parent in selected_parents:
        parent.tags["ps"] = True

    return population


def crossover(population: Population) -> Population:
    """
    Generational crossover with 100% reproduction rate.
    Creates offspring equal to target population size by selecting parents with replacement.
    """
    children = []

    # Get selected parents (those with ps=True from parent selection)
    parents = [ind for ind in population if ind.tags.get("ps", False)]

    # If no parents selected (shouldn't happen), use all population
    if len(parents) == 0:
        parents = population

    # Create target_population_size offspring
    for _ in range(EA_CONFIG.target_population_size):
        child = Individual()

        # Select two parents WITH replacement (can select same parent twice)
        parent1 = RNG.choice(parents)
        parent2 = RNG.choice(parents)

        # Generate a NEW random mask for every child
        shape = np.array(parent1.genotype).shape
        mask = RNG.random(size=shape) < 0.5

        # Perform uniform crossover
        child.genotype = np.where(
            mask,
            np.array(parent1.genotype),
            np.array(parent2.genotype),
        ).tolist()

        # Tags
        child.requires_eval = True
        child.tags["mut"] = True
        children.append(child)

    population.extend(children)
    return population


def mutation(population: Population) -> Population:
    """Mutates offspring by adding small random noise to genotype values."""
    mutation_rate = MUTATION_RATE
    
    # Debug: only print first 3 individuals to avoid spam
    # debug_count = 0
    # max_debug = 3
    
    for ind in population:
        if ind.tags.get("mut", False):
            # DEBUG: Show before mutation
            # if debug_count < max_debug:
                # print(f'\n--- Individual {debug_count + 1} ---')
            # print(f'Genotype BEFORE (first 5 genes): {ind.genotype[0][:5]}')
            # print(decode_genotype_to_string(ind.genotype))
            # print(decode_genotype_to_string(ind.genotype))
            # print(decode_genotype_to_string(ind.genotype))
            # print(decode_genotype_to_string(ind.genotype))
            # print(decode_genotype_to_string(ind.genotype))

            # print(f'Phenotype BEFORE: {before_string}')
            
            # Apply mutation
            genes = ind.genotype
            mutated = [
                float_creep(individual=genes[0], mutation_probability=mutation_rate),
                float_creep(individual=genes[1], mutation_probability=mutation_rate),
                float_creep(individual=genes[2], mutation_probability=mutation_rate),
            ]
            ind.genotype = mutated
            
            # DEBUG: Show after mutation
        # if debug_count < max_debug:
            # print(f'Genotype AFTER (first 5 genes): {ind.genotype[0][:5]}')
            # after_string = decode_genotype_to_string(ind.genotype)
            # print(f'Phenotype AFTER: {after_string}')
            
            # if before_string == after_string:
                # print('⚠️  WARNING: Phenotype unchanged!')
            # else:
                # print('✓ Phenotype changed')
                
                            
            ind.tags["mut"] = False
            ind.requires_eval = True

    return population


def fitness_tester(population: Population) -> Population:
    """Simple fitness function for testing: length of tree string."""
    for ind in population:
        if ind.requires_eval:
            # Use global NDE + new HPD (following A3_template pattern)
            matrixes = GLOBAL_NDE.forward(np.array(ind.genotype))
            hpd = HighProbabilityDecoder(num_modules=NUM_OF_MODULES)
            ind_graph = hpd.probability_matrices_to_graph(
                matrixes[0], matrixes[1], matrixes[2],
            )
            ind.tags["ctk_string"] = ctk.to_string(ctk.from_graph(ind_graph))
            ind.fitness = len(ind.tags["ctk_string"]) + 0.0
            ind.requires_eval = False

    return population


def survivor_selection_tournament(population: Population) -> Population:
    """
    K-tournament survivor selection with configurable tournament size.
    Higher tournament size = higher selection pressure.
    Repeatedly runs tournaments to eliminate worst individuals.
    """
    tournament_size = SURVIVOR_TOURNAMENT_SIZE
    current_pop_size = len([ind for ind in population if ind.alive])

    while current_pop_size > EA_CONFIG.target_population_size:
        # Get all alive individuals
        alive = [ind for ind in population if ind.alive]

        # Safety check
        if len(alive) <= EA_CONFIG.target_population_size:
            break

        # Randomly select tournament_size individuals for tournament
        competitors = RNG.choice(alive, size=min(tournament_size, len(alive)), replace=False)

        # Find the WORST individual in the tournament (to eliminate)
        if EA_CONFIG.is_maximisation:
            # In maximization, lowest fitness is worst
            loser = min(competitors, key=lambda ind: ind.fitness)
        else:
            # In minimization, highest fitness is worst
            loser = max(competitors, key=lambda ind: ind.fitness)

        # Eliminate the loser
        loser.alive = False
        current_pop_size -= 1

    return population

In [ ]:

def evaluate_diversity_cum_cos(population: Population) -> Population:
    # 1. Collect subtrees data
    for ind in population:
        if ind.requires_eval:
            ind.tags['ctk_string'] = decode_genotype_to_string(ind.genotype)
    
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(ind.tags['ctk_string']),
            config=SIM_CONFIG,
        ) for ind in population
    ]

    n_pop = len(population)
    keys = list(range(SIM_CONFIG.max_tree_radius))

    # Initialize the accumulator matrix (N x N)
    cumulative_sim_matrix = np.zeros((n_pop, n_pop))

    current_hasher = FeatureHasher(
        n_features=2**20,
        input_type="string",
    )

    # 2. Iterate keys and accumulate similarity matrices
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]

        # Transform is called repeatedly on the same object (Efficient)
        count_matrix = current_hasher.transform(specific_corpus)

        # Calculate Similarity
        sim_matrix = cosine_similarity(count_matrix)

        # Accumulate
        cumulative_sim_matrix += sim_matrix

    # 3. Average the matrix across all keys (radii)
    final_sim_matrix = cumulative_sim_matrix / len(keys)
    np.fill_diagonal(final_sim_matrix, 0)

    row_sums = final_sim_matrix.sum(axis=1)
    mean_similarity_to_others = row_sums / (n_pop - 1)
    diversity_scores = 1.0 - mean_similarity_to_others

    # 7. Assign to individuals
    for i, ind in enumerate(population):
        ind.fitness = float(diversity_scores[i])
        ind.requires_eval = False
        # nde = NeuralDevelopmentalEncoding(number_of_modules=NUM_OF_MODULES)
        # hpd = HighProbabilityDecoder(num_modules=NUM_OF_MODULES)
        # matrixes = nde.forward(np.array(ind.genotype))
        # ind_graph = hpd.probability_matrices_to_graph(
        #     matrixes[0], matrixes[1], matrixes[2],
        # )
        # ind.tags["ctk_string"] = ctk.to_string(ctk.from_graph(ind_graph))

    return population

In [ ]:
import numpy as np


def evaluate_novelty_cum_cos(population: Population) -> Population:
    global diversity_archive

    # 1. Collect features and calculate sizes
    current_subtrees = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(decode_genotype_to_string(ind.genotype)),
            config=SIM_CONFIG,
        ) for ind in population
    ]

    all_subtrees = diversity_archive + current_subtrees
    n_total = len(all_subtrees)
    n_pop = len(population)
    n_archive = len(diversity_archive)
    keys = list(range(SIM_CONFIG.max_tree_radius))

    # Handle initial empty state
    if n_total == 0:
        for ind in population:
            ind.fitness = 0.5
        diversity_archive.extend(current_subtrees)
        return population

    cumulative_sim_to_all = np.zeros(n_pop)

    current_hasher = FeatureHasher(
        n_features=2**20,
        input_type="string",
    )

    # --- Weighting Constants ---
    # We want the archive comparison to dominate the novelty score.
    # Current weight should be low (e.g., 5-10% max)
    WEIGHT_CURRENT = 0.10  # Reduced to a much lower weight
    WEIGHT_ARCHIVE = 1.0 - WEIGHT_CURRENT

    # 3. Calculate similarity for each radius key
    for key in keys:
        all_corpus = [d.get(key, []) for d in all_subtrees]

        # Transform all to raw counts (Sparse Matrix)
        all_features = current_hasher.transform(all_corpus)

        # CRITICAL FIX: Apply L2 Normalization (Required for Cosine Similarity)
        all_features = normalize(all_features, norm="l2", axis=1)

        # Split into archive and current (NOW normalized)
        if n_archive > 0:
            archive_features = all_features[:n_archive]
            current_features = all_features[n_archive:]

            # --- Similarity to Archive (The primary driver) ---
            sim_to_archive = cosine_similarity(current_features, archive_features)
            mean_sim_to_archive = sim_to_archive.mean(axis=1)  # Average sim of current individual to the entire archive

            # --- Similarity to Current (The low-weighted stabilizer) ---
            sim_to_current = cosine_similarity(current_features)
            np.fill_diagonal(sim_to_current, 0)  # Exclude self
            mean_sim_to_current = sim_to_current.sum(axis=1) / (n_pop - 1) if n_pop > 1 else np.zeros(n_pop)

            # Weighted average: Archive comparison dominates the score
            mean_sim = (WEIGHT_ARCHIVE * mean_sim_to_archive) + (WEIGHT_CURRENT * mean_sim_to_current)
        else:
            # Handle first generation case (only current vs current)
            current_features = all_features
            sim_to_current = cosine_similarity(current_features)
            np.fill_diagonal(sim_to_current, 0)
            mean_sim = sim_to_current.sum(axis=1) / (n_pop - 1) if n_pop > 1 else np.zeros(n_pop)

        cumulative_sim_to_all += mean_sim

    # 4. Average across keys and convert to diversity
    mean_similarity = cumulative_sim_to_all / len(keys)
    diversity_scores = 1.0 - mean_similarity

    # 5. Assign fitness
    for i, ind in enumerate(population):
        ind.fitness = float(diversity_scores[i])

    # 6. Add current population to archive (Correctly done)
    diversity_archive.extend(current_subtrees)

    # 7. Limit archive size (Correctly done)
    if len(diversity_archive) > MAX_ARCHIVE_SIZE:
        console.print(f"ARCHIVE MAX SIZE REACHED {len(global_fitness_history)}")
        diversity_archive = diversity_archive[-MAX_ARCHIVE_SIZE:]

    return population

In [ ]:

def evaluate_diversity_cum_cos(population: Population) -> Population:
    # 1. Collect subtrees data
    for ind in population:
        if ind.requires_eval:
            ind.tags['ctk_string'] = decode_genotype_to_string(ind.genotype)
    
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(ind.tags['ctk_string']),
            config=SIM_CONFIG,
        ) for ind in population
    ]

    # console.print(subtrees_dicts)

    n_pop = len(population)
    keys = list(range(SIM_CONFIG.max_tree_radius + 1))

    # console.print(keys)

    # Initialize the accumulator matrix (N x N)
    cumulative_sim_matrix = np.zeros((n_pop, n_pop))

    current_hasher = FeatureHasher(
        n_features=2**20,
        input_type="string",
    )

    # 2. Iterate keys and accumulate similarity matrices
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]

        # console.print(specific_corpus)

        # Transform is called repeatedly on the same object (Efficient)
        count_matrix = current_hasher.fit_transform(specific_corpus)

        # console.print(count_matrix.toarray())

        # Calculate Similarity
        sim_matrix = cosine_similarity(count_matrix)

        # console.print(sim_matrix)

        # Accumulate
        cumulative_sim_matrix += sim_matrix

    # 3. Average the matrix across all keys (radii)
    final_sim_matrix = cumulative_sim_matrix / len(keys)
    np.fill_diagonal(final_sim_matrix, 0)

    # console.print(final_sim_matrix)

    row_sums = final_sim_matrix.sum(axis=1)

    # console.print('row sums', row_sums)

    mean_similarity_to_others = row_sums / (n_pop - 1)

    # console.print('dev', (n_pop - 1))

    # console.print('mean similarity to others', mean_similarity_to_others)

    diversity_scores = 1.0 - mean_similarity_to_others

    # console.print('diversity scores', diversity_scores)

    # 7. Assign to individuals
    for i, ind in enumerate(population):
        ind.fitness = float(diversity_scores[i])
        ind.requires_eval = False

        # nde = NeuralDevelopmentalEncoding(number_of_modules=NUM_OF_MODULES)
        # hpd = HighProbabilityDecoder(num_modules=NUM_OF_MODULES)
        # matrixes = nde.forward(np.array(ind.genotype))
        # ind_graph = hpd.probability_matrices_to_graph(
        #     matrixes[0], matrixes[1], matrixes[2],
        # )
        # ind.tags["ctk_string"] = ctk.to_string(ctk.from_graph(ind_graph))

    return population

In [ ]:
import numpy as np


def evaluate_diversity_tfidf_cos(population: Population) -> Population:
    # 1. Collect subtrees data
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(decode_genotype_to_string(ind.genotype)),
            config=SIM_CONFIG,
        ) for ind in population
    ]

    # console.print(subtrees_dicts)

    n_pop = len(population)
    # Corrected keys: range(max_tree_radius + 1)
    keys = list(range(SIM_CONFIG.max_tree_radius + 1))

    # console.print(keys)

    cumulative_sim_matrix = np.zeros((n_pop, n_pop))

    # Initialize FeatureHasher and TF-IDF Transformer ONCE
    current_hasher = FeatureHasher(
        n_features=2**20,
        input_type="string",
    )
    # The TfidfTransformer will learn the IDF based on the current population's raw counts
    tfidf_transformer = TfidfTransformer(use_idf=True)

    # 2. Iterate keys and accumulate similarity matrices
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]

        # console.print(specific_corpus)

        # --- Step A: Hash to get Raw Counts (CSR Matrix) ---
        count_matrix = current_hasher.fit_transform(specific_corpus)

        # --- Step B: Apply TF-IDF Transformation (Required .fit_transform) ---
        # This converts raw counts into weighted features, penalizing common subtrees.
        # This must be done inside the loop to calculate IDF based on the current population.
        weighted_matrix = tfidf_transformer.fit_transform(count_matrix)

        # --- Step C: Apply L2 Normalization (Required for Cosine Similarity) ---
        # The TfidfTransformer may include L2 norm if norm='l2' is set in its constructor.
        # However, for safety and clarity when mixing tools, we explicitly normalize.
        normalized_matrix = normalize(weighted_matrix, norm="l2", axis=1)

        # --- Step D: Calculate Similarity (on normalized vectors) ---
        sim_matrix = cosine_similarity(normalized_matrix)  # Equivalent to normalized_matrix @ normalized_matrix.T

        # console.print(sim_matrix)

        # Accumulate
        cumulative_sim_matrix += sim_matrix

    # 3. Average the matrix across all keys (radii)
    final_sim_matrix = cumulative_sim_matrix / len(keys)
    np.fill_diagonal(final_sim_matrix, 0)

    # console.print(final_sim_matrix)

    row_sums = final_sim_matrix.sum(axis=1)
    mean_similarity_to_others = row_sums / (n_pop - 1)
    diversity_scores = 1.0 - mean_similarity_to_others

    # 7. Assign to individuals
    for i, ind in enumerate(population):
        ind.fitness = float(diversity_scores[i])

    return population

In [ ]:
def evaluate_diversity_cum_cos_archive(population: Population) -> Population:
    """
    Archive-based novelty search: compares current individuals to archive.
    Novelty = dissimilarity to archive (not to current population).
    NOTE: Does NOT add to archive - that's handled by add_survivors_to_archive().
    """
    global diversity_archive

    # 1. Collect current population's subtrees
    current_subtrees = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(decode_genotype_to_string(ind.genotype)),
            config=SIM_CONFIG,
        ) for ind in population
    ]

    n_pop = len(population)
    n_archive = len(diversity_archive)
    keys = list(range(SIM_CONFIG.max_tree_radius + 1))

    # Handle first generation (no archive yet)
    if n_archive == 0:
        console.print("First generation: no archive, assigning neutral fitness")
        for ind in population:
            ind.fitness = 0.5  # Neutral novelty
        return population

    # Initialize similarity accumulator for each individual
    cumulative_similarity_to_archive = np.zeros(n_pop)

    current_hasher = FeatureHasher(
        n_features=2**20,
        input_type="string",
    )

    # 2. For each tree radius, calculate similarity to archive
    for key in keys:
        # Get features for archive and current population
        archive_corpus = [d.get(key, []) for d in diversity_archive]
        current_corpus = [d.get(key, []) for d in current_subtrees]

        # Transform archive and current separately
        # NOTE: FeatureHasher doesn't need fitting, it's stateless
        archive_features = current_hasher.transform(archive_corpus)
        current_features = current_hasher.transform(current_corpus)

        # Calculate similarity: current vs archive
        # Result shape: (n_pop, n_archive)
        # Each row = one current individual's similarity to all archive individuals
        sim_to_archive = cosine_similarity(current_features, archive_features)

        # For each current individual, take mean similarity to archive
        mean_sim_per_individual = sim_to_archive.mean(axis=1)  # Shape: (n_pop,)

        # Accumulate across tree radii
        cumulative_similarity_to_archive += mean_sim_per_individual

    # 3. Average across all tree radii
    mean_similarity_to_archive = cumulative_similarity_to_archive / len(keys)

    # 4. Convert similarity to novelty (dissimilarity)
    novelty_scores = 1.0 - mean_similarity_to_archive

    console.print(f"Archive size: {n_archive}, Mean novelty: {novelty_scores.mean():.4f}")

    # 5. Assign fitness (higher novelty = higher fitness)
    for i, ind in enumerate(population):
        ind.fitness = float(novelty_scores[i])

    # NOTE: Archive update removed - handled by add_survivors_to_archive()

    return population

In [ ]:
import numpy as np

# Assuming K_NEIGHBORS is defined globally, e.g.,


def fitness_cos_knn(population: Population) -> Population:
    # --- Setup ---
    K = K_NEIGHBORS  # Use the global K
    
    for ind in population:
        if ind.requires_eval:
            ind.tags['ctk_string'] = decode_genotype_to_string(ind.genotype)
    
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(ind.tags['ctk_string']),
            config=SIM_CONFIG,
        ) for ind in population
    ]
    
    # subtrees_dicts = [
    #     ctk.collect_tree_hash_config_mode(
    #         ctk.from_string(decode_genotype_to_string(ind.genotype)),
    #         config=SIM_CONFIG,
    #     ) for ind in population
    # ]

    n_pop = len(population)
    keys = list(range(SIM_CONFIG.max_tree_radius + 1))
    cumulative_sim_matrix = np.zeros((n_pop, n_pop))

    current_hasher = FeatureHasher(n_features=2**20, input_type="string")

    # 1. Accumulate Similarity Matrix
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]

        # Hash to get Raw Counts
        count_matrix = current_hasher.transform(specific_corpus)

        # Apply L2 Normalization (Required for Cosine Similarity)
        normalized_matrix = normalize(count_matrix, norm="l2", axis=1)

        # Calculate Similarity
        sim_matrix = cosine_similarity(normalized_matrix)

        # Accumulate
        cumulative_sim_matrix += sim_matrix

    # 2. Final Averaging (Average similarity across all keys)
    final_sim_matrix = cumulative_sim_matrix / len(keys)

    # 3. CRITICAL STEP: Convert Similarity to Distance/Dissimilarity
    # Distance = 1 - Similarity. This is the required input for kNN distance summation.
    distance_matrix = 1.0 - final_sim_matrix

    # 4. Calculate K-Nearest Neighbor Novelty Score (Raw Sum)

    N = distance_matrix.shape[0]

    if N <= K:
        # If population size is too small, use all available neighbors (excluding self)
        k_actual = N - 1
    else:
        k_actual = K

    if k_actual == 0:
        # Avoid division by zero/zero summation if population is size 1
        novelty_scores_sum = np.zeros(N)
    else:
        # 4a. Find the k+1 smallest distances in each row (including self-distance=0)
        # argpartition is used for efficiency.
        k_plus_1_indices = np.argpartition(distance_matrix, k_actual + 1, axis=1)

        # 4b. Select the indices of the actual k nearest neighbors (skipping index 0)
        knn_indices = k_plus_1_indices[:, 1:k_actual + 1]

        # 4c. Gather the actual distance values
        knn_distances = np.take_along_axis(distance_matrix, knn_indices, axis=1)

        # 4d. Collapse: Sum the k distances for each row (Raw Novelty Score)
        novelty_scores_sum = knn_distances.sum(axis=1)

    # 5. Assign to individuals (Raw sum is the fitness score)
    for i, ind in enumerate(population):
        # We maximize the raw sum of distances (higher sum = higher novelty/fitness)
        ind.fitness = float(novelty_scores_sum[i])

    return population

In [ ]:
import numpy as np

# Note: You must ensure 'umap' is imported and accessible.

# Assuming the global K_NEIGHBORS (for fitness) and GLOBAL_N_NEIGHBORS (for UMAP internal k) are defined.


def evaluate_umap_knn_fitness(population: Population) -> Population:
    """
    Calculates Novelty Fitness based on the sum of Euclidean distances to K nearest
    neighbors in the UMAP-transformed space (Maximization).
    """
    # Use global K values
    K = K_NEIGHBORS
    K_UMAP_INTERNAL = 2

    # --- 1. Setup & Data Collection ---
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(decode_genotype_to_string(ind.genotype)),
            config=SIM_CONFIG,
        ) for ind in population
    ]

    n_pop = len(population)
    # Corrected keys: range(max_tree_radius + 1)
    keys = list(range(SIM_CONFIG.max_tree_radius + 1))

    # Initialize accumulator for the distance matrix (not similarity)
    # We will accumulate distance matrices, not similarity matrices.
    cumulative_dist_matrix = np.zeros((n_pop, n_pop))

    # Initialize shared transformers
    current_hasher = FeatureHasher(n_features=2**20, input_type="string")
    tfidf_transformer = TfidfTransformer(use_idf=True)  # Recommended for structure

    # 2. Accumulate Distance Matrix (Iterate over radii)
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]

        # --- A. Hashing & Weighting (Hash -> TF-IDF) ---
        count_matrix = current_hasher.transform(specific_corpus)
        # Using TF-IDF to focus on unique structure, penalizing common subtrees
        weighted_matrix = tfidf_transformer.fit_transform(count_matrix)

        # --- B. Dimensionality Reduction (UMAP) ---
        # UMAP transforms the high-dim weighted vector into a low-dim embedding
        umap_model = umap.UMAP(
            init="random",
            random_state=42,
            transform_seed=42,
            n_jobs=1,
            metric="cosine",  # Use cosine distance in the high-dim space for graph construction
            n_neighbors=K_UMAP_INTERNAL,
        )
        umap_embeddings = umap_model.fit_transform(weighted_matrix)

        # --- C. Distance Calculation (Euclidean in UMAP space) ---
        # pdist generates condensed distance vector; squareform converts to N x N matrix.
        condensed_distances = pdist(umap_embeddings, metric="euclidean")
        distance_matrix = squareform(condensed_distances)

        # Accumulate the distance matrices
        cumulative_dist_matrix += distance_matrix

    # 3. Final Averaging (Average distance across all keys)
    final_dist_matrix = cumulative_dist_matrix / len(keys)

    # 4. Calculate K-Nearest Neighbor Novelty Score (Raw Sum)

    N = final_dist_matrix.shape[0]

    k_actual = N - 1 if N <= K else K

    if k_actual == 0:
        novelty_scores_sum = np.zeros(N)
    else:
        # 4a. Find the k+1 smallest distances in each row (including self-distance=0)
        k_plus_1_indices = np.argpartition(final_dist_matrix, k_actual + 1, axis=1)

        # 4b. Select the indices of the actual k nearest neighbors (skipping index 0)
        knn_indices = k_plus_1_indices[:, 1:k_actual + 1]

        # 4c. Gather the actual distance values
        knn_distances = np.take_along_axis(final_dist_matrix, knn_indices, axis=1)

        # 4d. Collapse: Sum the k distances for each row (Raw Novelty Score)
        novelty_scores_sum = knn_distances.sum(axis=1)

    # 5. Assign to individuals (Raw sum is the fitness score)
    for i, ind in enumerate(population):
        # Higher sum of distances means higher novelty/fitness
        ind.fitness = float(novelty_scores_sum[i])

    return population

In [ ]:
import numpy as np
from scipy.sparse import hstack  # REQUIRED for horizontal stacking


def umap_aggregate_single_pass(population: Population) -> Population:

    # Use global K values
    K = K_NEIGHBORS
    K_UMAP_INTERNAL = 2

    # --- 1. Data Collection & Hashing ---
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(ctk.from_string(decode_genotype_to_string(ind.genotype)), config=SIM_CONFIG)
        for ind in population
    ]

    len(population)
    keys = list(range(SIM_CONFIG.max_tree_radius + 1))

    # Initialize shared transformers
    current_hasher = FeatureHasher(n_features=2**21, input_type="string")
    tfidf_transformer = TfidfTransformer(use_idf=True)

    # List to hold the weighted feature matrix for each radius (to be stacked)
    weighted_feature_list = []

    # 2. Consolidated Feature Extraction & Weighting
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]

        # Hash to get Raw Counts
        count_matrix = current_hasher.transform(specific_corpus)

        # Apply TF-IDF (This step MUST be inside the loop to calculate IDF per key)
        weighted_matrix = tfidf_transformer.fit_transform(count_matrix)

        weighted_feature_list.append(weighted_matrix)

    # 3. Aggregate Features (Horizontal Stack)
    # The final matrix now has shape (N_pop x Total_Features)
    final_feature_matrix = hstack(weighted_feature_list)

    # 4. Dimensionality Reduction (UMAP - Single Pass)
    umap_model = umap.UMAP(
        init="random",
        random_state=42,
        transform_seed=42,
        n_jobs=1,
        metric="cosine",  # Metric on the high-dimensional space
        n_neighbors=K_UMAP_INTERNAL,
    )
    # Fit and transform the entire consolidated feature set
    umap_embeddings = umap_model.fit_transform(final_feature_matrix)

    # 5. Distance Calculation (Euclidean in UMAP space)
    condensed_distances = pdist(umap_embeddings, metric="euclidean")
    distance_matrix = squareform(condensed_distances)

    # 6. Calculate K-Nearest Neighbor Novelty Score (Raw Sum)

    N = distance_matrix.shape[0]

    k_actual = N - 1 if N <= K else K

    if k_actual == 0:
        novelty_scores_sum = np.zeros(N)
    else:
        k_plus_1_indices = np.argpartition(distance_matrix, k_actual + 1, axis=1)
        knn_indices = k_plus_1_indices[:, 1:k_actual + 1]
        knn_distances = np.take_along_axis(distance_matrix, knn_indices, axis=1)
        novelty_scores_sum = knn_distances.sum(axis=1)

    # 7. Assign to individuals
    for i, ind in enumerate(population):
        ind.fitness = float(novelty_scores_sum[i])

    return population

In [ ]:
import glob
import os

import numpy as np
from scipy.sparse import csr_matrix, load_npz, save_npz, vstack


def save_generation_features(
    sparse_matrix: csr_matrix,
    generation_number: int,
    # DEFAULT IS SET HERE
    base_path: str = "./__data__/archive/features",
) -> None:
    """Saves the sparse feature matrix for the current generation quickly."""
    # Ensure the directory exists before attempting to save
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    filename = os.path.join(base_path, f"features_g{generation_number:04d}.npz")
    save_npz(filename, sparse_matrix)

    # Note: You would update the other function signatures (like load_and_stack_all_features)
    # to also use this same default path.


def load_and_stack_all_features(base_path: str) -> csr_matrix | None:
    """Loads all historical sparse feature files and stacks them into one matrix."""
    file_list = sorted(glob.glob(os.path.join(base_path, "features_g*.npz")))

    if not file_list:
        return None

    all_matrices = [load_npz(f) for f in file_list]

    # Vertically stack all generations into one massive sparse matrix
    return vstack(all_matrices)

In [ ]:
# Assuming current_gen_count is available (e.g., from a global tracker)
def extract_and_save_current_features(population: Population, current_gen_count: int, base_path: str):
    """Hashes the current population's subtrees and saves the raw count matrix."""
    # 1. Collect features (similar to your previous setup)
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(ctk.from_string(decode_genotype_to_string(ind.genotype)), config=SIM_CONFIG)
        for ind in population
    ]
    keys = list(range(SIM_CONFIG.max_tree_radius + 1))

    # Initialize Hasher (Only needs to happen once for the raw features)
    current_hasher = FeatureHasher(n_features=2**20, input_type="string")

    # List to hold the raw feature matrix for each radius (to be stacked horizontally)
    raw_feature_list = []

    # 2. Consolidated Feature Extraction (No TF-IDF or L2 yet!)
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]
        count_matrix = current_hasher.transform(specific_corpus)
        raw_feature_list.append(count_matrix)

    # 3. Aggregate Features (Horizontal Stack)
    raw_feature_matrix_gen = hstack(raw_feature_list)

    # 4. Save the raw feature matrix for this generation
    save_generation_features(raw_feature_matrix_gen, current_gen_count, base_path)

    return population

In [ ]:
import numpy as np
from scipy.sparse import issparse


def fitness_rarity_magnitude(population: Population) -> Population:
    """
    Calculates fitness based on the L2 Norm (Magnitude) of the aggregated,
    TF-IDF weighted feature vector for each individual (Rarity Magnitude Search).
    """
    # --- 1. Setup & Data Collection ---
    subtrees_dicts = [
        ctk.collect_tree_hash_config_mode(
            ctk.from_string(decode_genotype_to_string(ind.genotype)),
            config=SIM_CONFIG,
        ) for ind in population
    ]

    keys = list(range(SIM_CONFIG.max_tree_radius + 1))

    # Initialize shared transformers
    current_hasher = FeatureHasher(n_features=2**20, input_type="string")
    tfidf_transformer = TfidfTransformer(use_idf=True, norm="l2")  # Ensure final L2 norm is applied

    weighted_feature_list = []

    # 2. Consolidated Feature Extraction & Weighting
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]

        # Hash to get Raw Counts
        count_matrix = current_hasher.transform(specific_corpus)

        # Apply TF-IDF (This step calculates the final L2 norm on the feature vector)
        # We use .fit_transform here to correctly learn the IDF based on the current population.
        weighted_matrix = tfidf_transformer.fit_transform(count_matrix)

        weighted_feature_list.append(weighted_matrix)

    # 3. Aggregate Features (Horizontal Stack)
    # The final matrix now has shape (N_pop x Total_Features)
    final_feature_matrix = hstack(weighted_feature_list)

    # 4. Calculate the Rarity Magnitude (L2 Norm)

    # The L2 norm of a sparse matrix is calculated efficiently using .power(2) and .sum(axis=1)
    if issparse(final_feature_matrix):
        # Calculate the sum of squares for each row
        # (This is equivalent to the square of the L2 norm)
        square_norms = final_feature_matrix.power(2).sum(axis=1)

        # Calculate the square root to get the final L2 norm
        rarity_magnitude_scores = np.sqrt(square_norms).A.flatten()
    else:
        # Fallback for dense matrix (less likely here)
        rarity_magnitude_scores = np.linalg.norm(final_feature_matrix, axis=1)

    # 5. Assign to individuals (Fitness = Rarity Magnitude)
    for i, ind in enumerate(population):
        # Higher magnitude means higher rarity and complexity, maximizing the score.
        ind.fitness = float(rarity_magnitude_scores[i])

    return population

In [ ]:
import pathlib
import shutil

# Global state variables
GLOBAL_GENERATION_COUNT = 0
GLOBAL_RAW_HISTORY = None
GLOBAL_TFIDF = TfidfTransformer(use_idf=True)  # FIXED: Removed norm='l2' to avoid double normalization


def clear_archive(base_path="./__data__/archive/features") -> None:
    """Deletes the archive directory to start fresh."""
    if pathlib.Path(base_path).exists():
        shutil.rmtree(base_path)
        console.print(f"[yellow]Cleared archive at: {base_path}[/yellow]")
    else:
        console.print(f"[dim]No archive found at: {base_path}[/dim]")


def fitness_online_tfidf_archive(population: Population) -> Population:
    """
    Calculates fitness using incremental TF-IDF and updates the archive.
    Manages generation counting internally.
    Auto-clears archive on first generation (gen 0).
    Uses GLOBAL_NDE for deterministic decoding (following A3_template pattern).
    """
    # Bring in the globals to track state across generations
    global GLOBAL_RAW_HISTORY, GLOBAL_TFIDF, GLOBAL_GENERATION_COUNT

    # Define the path (hardcoded here or use a global config constant)
    base_path = "./__data__/archive/features"

    # Auto-clear archive on first generation to prevent contamination from previous runs
    if GLOBAL_GENERATION_COUNT == 0:
        clear_archive(base_path)

    # --- 1. Extract Raw Features for Current Population ---
    # FIXED: Changed genotype_ to genotype
    # FIXED: Using GLOBAL_NDE + new HPD each time (A3_template pattern)
    subtrees_dicts = []
    for ind in population:
        matrixes = GLOBAL_NDE.forward(np.array(ind.genotype))
        hpd = HighProbabilityDecoder(num_modules=NUM_OF_MODULES)
        ind_graph = hpd.probability_matrices_to_graph(matrixes[0], matrixes[1], matrixes[2])
        tree = ctk.from_graph(ind_graph)
        subtree_dict = ctk.collect_tree_hash_config_mode(tree, config=SIM_CONFIG)
        subtrees_dicts.append(subtree_dict)

    keys = list(range(SIM_CONFIG.max_tree_radius + 1))
    current_hasher = FeatureHasher(n_features=2**20, input_type="string")

    raw_feature_list = []
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]
        count_matrix = current_hasher.transform(specific_corpus)
        raw_feature_list.append(count_matrix)

    # The raw counts for THIS generation
    current_raw_matrix = hstack(raw_feature_list)

    # --- 2. Combine History + Current for Fitting ---
    if GLOBAL_RAW_HISTORY is not None:
        # Stack history on top of current to create the full corpus
        full_corpus_for_fitting = vstack([GLOBAL_RAW_HISTORY, current_raw_matrix])
    else:
        # First generation: History is just the current population
        full_corpus_for_fitting = current_raw_matrix

    # --- 3. Refit TF-IDF on the Full Corpus ---
    # This learns the IDF weights based on EVERYTHING found so far.
    GLOBAL_TFIDF.fit(full_corpus_for_fitting)

    # --- 4. Transform the ACTUAL (Current) Matrix ---
    # Apply the newly learned weights to the current population
    current_weighted_matrix = GLOBAL_TFIDF.transform(current_raw_matrix)

    # --- 5. Calculate Fitness (L2 Norm / Rarity Magnitude) ---
    if issparse(current_weighted_matrix):
        square_norms = current_weighted_matrix.power(2).sum(axis=1)
        rarity_scores = np.sqrt(square_norms).A.flatten()
    else:
        rarity_scores = np.linalg.norm(current_weighted_matrix, axis=1)

    for i, ind in enumerate(population):
        ind.fitness = float(rarity_scores[i])

    # --- 6. Update the Archive (Disk & Memory) ---

    # A. Save to disk using the Global Generation Count
    save_generation_features(current_raw_matrix, GLOBAL_GENERATION_COUNT, base_path)

    # B. Update Memory
    GLOBAL_RAW_HISTORY = full_corpus_for_fitting

    # C. Increment Generation Count for next time
    GLOBAL_GENERATION_COUNT += 1

    return population

In [ ]:
GLOBAL_GENERATION_COUNT = 0
def save_featues(population: Population):
    global GLOBAL_GENERATION_COUNT 
    GLOBAL_GENERATION_COUNT += 1
    base_path = "./__data__/archive/features"    
    
    subtrees_dicts = []
    for ind in population:
        matrixes = GLOBAL_NDE.forward(np.array(ind.genotype))
        hpd = HighProbabilityDecoder(num_modules=NUM_OF_MODULES)
        ind_graph = hpd.probability_matrices_to_graph(matrixes[0], matrixes[1], matrixes[2])
        tree = ctk.from_graph(ind_graph)
        subtree_dict = ctk.collect_tree_hash_config_mode(tree, config=SIM_CONFIG)
        subtrees_dicts.append(subtree_dict)

    keys = list(range(SIM_CONFIG.max_tree_radius + 1))
    current_hasher = FeatureHasher(n_features=2**20, input_type="string")

    raw_feature_list = []
    for key in keys:
        specific_corpus = [d.get(key, []) for d in subtrees_dicts]
        count_matrix = current_hasher.transform(specific_corpus)
        raw_feature_list.append(count_matrix)

    current_raw_matrix = hstack(raw_feature_list)    
    save_generation_features(current_raw_matrix, GLOBAL_GENERATION_COUNT, base_path)
    return population

# run


In [ ]:
population_list = [
    make_random_robot(GENOTYPE_SIZE) for _ in range(EA_CONFIG.target_population_size)
]


In [ ]:
ops = [
    EAStep("parent_selection", parent_selection_tournament),  # K-tournament (size=3)
    EAStep("crossover", crossover),
    EAStep("mutation", mutation),

    
    # EAStep('diveristy', evaluate_diversity_cum_cos),
    # EAStep("tfidf_archive", fitness_online_tfidf_archive),
    # EAStep('umap_agg', umap_aggregate_single_pass),
    # EAStep('umap', evaluate_umap_knn_fitness),
    # EAStep('fitness', fitness_tester),
    EAStep("evaluation", fitness_cos_knn),
    # EAStep("evaluation", evaluate_diversity_tfidf_cos),
    # EAStep("evaluation", evaluate_novelty_cum_cos),
    EAStep("survivor_selection", survivor_selection_tournament),  # K-tournament (size=3)
    # EAStep("archive_survivors", add_survivors_to_archive),  # NEW: Archive only survivors
    
    # EAStep('save_featues', save_featues),
    EAStep("record_fitness", record_mean_fitness),
    EAStep("best_fitness", record_best_fitness),
]

In [ ]:
ea = EA(
    population_list,
    operations=ops,
    num_of_generations=EA_CONFIG.num_of_generations,
)

In [ ]:
ea.run()

In [ ]:
def plot_fitness_history() -> None:
    if not global_fitness_history:
        return

    generations = range(1, len(global_fitness_history) + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(
        generations,
        global_fitness_history,
        # marker="o",
        linestyle="-",
        color="blue",
        label="mean",
    )
    plt.plot(
        generations,
        global_best_fitness_history,
        # marker="o",
        linestyle="-",
        color="red",
        label="best",
    )

    y_max1 = np.max(global_fitness_history)
    y_max2 = np.max(global_best_fitness_history)

    y_max = np.max([y_max1, y_max2])

    # --- THE FIX ---
    # This forces the Y-axis to start at 0.
    # Since your data is normalized (0 to 1), setting the top to 1.1 is also usually good practice.
    # plt.ylim(0, y_max * 1.2)
    # ----------------

    plt.legend(loc="upper right")
    # plt.title("")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.show()


plot_fitness_history()

In [ ]:
import numpy as np
from bokeh.models import ColorBar, ColumnDataSource, HoverTool
from bokeh.palettes import Turbo256  # Good palettes for time depth
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap


def plot_archive_time_lapse(
    umap_coordinates: np.ndarray,
    generation_ids: np.ndarray,
    images: list | None = None,
    title: str = "Evolutionary Time-Lapse",
    plot_width: int = 800,
    plot_height: int = 600,
    point_size: int = 6,
    alpha: float = 0.6,
    selected_generations: list | None = None,  # NEW: Filter specific generations
) -> None:
    """
    Plots the entire evolutionary history in a single UMAP space.
    Color indicates Generation Number (Time).

    Args:
        umap_coordinates: (N_total, 2) array of UMAP embeddings for the archive.
        generation_ids: (N_total,) array of generation numbers corresponding to each point.
        images: Optional list of base64 images (len = N_total). If None, tooltips show only ID/Gen.
        selected_generations: Optional list of generation numbers to visualize.
                            If None, all generations are plotted.
                            Examples: [0, 1, 2] or range(10, 20) or [0, 25, 49]
    """
    # Filter data if specific generations are requested
    if selected_generations is not None:
        selected_generations = list(selected_generations)  # Convert range to list if needed
        mask = np.isin(generation_ids, selected_generations)

        umap_coordinates = umap_coordinates[mask]
        generation_ids = generation_ids[mask]

        if images is not None:
            images = [img for i, img in enumerate(images) if mask[i]]

        console.print(f"[cyan]Filtered to {len(selected_generations)} generations: {sorted(selected_generations)}[/cyan]")
        console.print(f"[cyan]Plotting {mask.sum()} individuals[/cyan]")

    n_total = len(generation_ids)

    if n_total == 0:
        return

    # 1. Setup Color Mapper (Time = Color)
    # We use linear_cmap to map the 'generation' column directly to colors
    # Turbo256 is great because it has high contrast (Red=New, Blue=Old)
    color_mapper = linear_cmap(
        field_name="generation",
        palette=Turbo256,
        low=min(generation_ids),
        high=max(generation_ids),
    )

    # 2. Build DataFrame
    data = {
        "x": umap_coordinates[:, 0],
        "y": umap_coordinates[:, 1],
        "generation": generation_ids,
        "id": np.arange(n_total),
    }

    # 3. Handle Optional Images
    if images is not None:
        if len(images) != n_total:
            tooltip_html = """
                <div style="font-size:14px; font-weight: bold;">Gen: @generation</div>
                <div style="font-size:12px; color:#666;">ID: @id</div>
            """
        else:
            data["image"] = images
            tooltip_html = """
                <div><img src='@image' style='float:left; margin:5px; width:100px; height:auto;'/></div>
                <div style="font-size:14px; font-weight: bold;">Gen: @generation</div>
                <div style="font-size:12px; color:#666;">ID: @id</div>
            """
    else:
        tooltip_html = """
            <div style="font-size:14px; font-weight: bold;">Gen: @generation</div>
            <div style="font-size:12px; color:#666;">ID: @id</div>
        """

    source = ColumnDataSource(data)

    # 4. Create Plot
    p = figure(
        title=title,
        width=plot_width,
        height=plot_height,
        tools="pan,wheel_zoom,reset,save",
        toolbar_location="above",
        match_aspect=True,  # Keeps UMAP aspect ratio correct
    )

    # 5. Scatter Layer
    # Sort by generation so new points (bright) are drawn ON TOP of old points
    # Note: Bokeh doesn't have simple z-order sorting in CDS, so we usually rely on input order.
    # If your archive is already sorted by time, this is automatic.

    renderer = p.scatter(
        "x", "y",
        source=source,
        size=point_size,
        fill_color=color_mapper,  # The magic time coloring
        line_color=None,         # No border makes clusters clearer
        fill_alpha=alpha,
    )

    # 6. Add Color Bar (Legend)
    color_bar = ColorBar(
        color_mapper=color_mapper["transform"],
        width=8,
        location=(0, 0),
        title="Generation",
        title_text_font_size="10pt",
    )
    p.add_layout(color_bar, "right")

    # 7. Add Tooltip
    hover = HoverTool(tooltips=tooltip_html, renderers=[renderer])
    p.add_tools(hover)

    # Optional: Clean up axes
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.xgrid.visible = False
    p.ygrid.visible = False
    p.background_fill_color = "#fafafa"

    show(p)

In [ ]:
import numpy as np


def load_archive_and_generate_ids(base_path="./__data__/archive/features"):
    """
    Loads all .npz files, stacks them, and generates the corresponding
    generation_id array based on the actual file contents.
    """
    # 1. Find and sort files (e.g., features_g0000.npz, features_g0001.npz)
    file_list = sorted(glob.glob(os.path.join(base_path, "features_g*.npz")))

    if not file_list:
        msg = "No feature files found!"
        raise FileNotFoundError(msg)

    matrices = []
    gen_ids_list = []

    for filepath in file_list:
        # Extract generation number from filename "features_g0050.npz"
        # Split by '_g', take the last part, remove '.npz', convert to int
        try:
            filename = pathlib.Path(filepath).name
            gen_num = int(filename.split("_g")[1].split(".")[0])
        except (IndexError, ValueError):
            continue

        # Load the matrix
        matrix = load_npz(filepath)
        matrices.append(matrix)

        # Create an ID array of the same length as this generation's population
        # shape[0] is the number of individuals in that specific generation
        current_ids = np.full(matrix.shape[0], gen_num)
        gen_ids_list.append(current_ids)

    # 2. Stack everything
    F_total = vstack(matrices)
    all_gen_ids = np.concatenate(gen_ids_list)

    return F_total, all_gen_ids

In [ ]:
import sys

import numpy as np
import umap.plot
from bokeh.io import output_notebook
from rich.console import Console
from sklearn.feature_extraction.text import TfidfTransformer

from ariel_experiments.characterize.canonical.core.toolkit import (
    CanonicalToolKit as ctk,
)

console = Console()
output_notebook()

In [ ]:
raise KeyboardInterrupt

In [ ]:
# --- CONFIGURATION ---
ARCHIVE_PATH = "./__data__/archive/features"
K_NEIGHBORS = 15  # Same K you used for fitness

# 1. Load Data
try:
    F_total, generation_ids = load_archive_and_generate_ids(ARCHIVE_PATH)
except Exception:
    sys.exit()

# 2. Run UMAP (The Final Transformation)
# We fit on the ENTIRE history to get the global coordinate system

# Important: Use the same metric you used during evolution (usually cosine)
umap_reducer = umap.UMAP(
    n_components=10,
    metric="cosine",
    n_neighbors=2,
    min_dist=0.1,
    random_state=42,
)

# Get the 2D coordinates
all_umap_coordinates = umap_reducer.fit_transform(F_total)

# 3. Plot

# (Assuming 'plot_archive_time_lapse' is defined in your scope)
plot_archive_time_lapse(
    umap_coordinates=all_umap_coordinates,
    generation_ids=generation_ids,
    images=None,  # Set to None for pure data visualization
    title=f"Evolutionary Trajectory (Gen 0 - {max(generation_ids)})",
    alpha=0.6,   # Lower alpha helps see clusters
    point_size=4,  # Smaller points for large archives
)

In [ ]:
plot_archive_time_lapse(
      umap_coordinates=all_umap_coordinates,
      generation_ids=generation_ids,
  )

# # 2. Plot specific generations (list):
# # Plot only generations 0, 10, 20, 30, 40, 49
# plot_archive_time_lapse(
#     umap_coordinates=all_umap_coordinates,
#     generation_ids=generation_ids,
#     selected_generations=[0, 10, 20, 30, 40, 49],
#     title="Selected Generations (0, 10, 20, 30, 40, 49)",
# )

# # 3. Plot a range of generations:
# # Plot first 10 generations
# plot_archive_time_lapse(
#     umap_coordinates=all_umap_coordinates,
#     generation_ids=generation_ids,
#     selected_generations=range(10),
#     title="First 10 Generations",
# )

# # Plot generations 40-49
# plot_archive_time_lapse(
#     umap_coordinates=all_umap_coordinates,
#     generation_ids=generation_ids,
#     selected_generations=range(40, 50),
#     title="Final 10 Generations (40-49)",
# )

# # 4. Compare early vs late:
# # Plot every 10th generation
# plot_archive_time_lapse(
#     umap_coordinates=all_umap_coordinates,
#     generation_ids=generation_ids,
#     selected_generations=range(0, 50, 10),  # 0, 10, 20, 30, 40
#     title="Every 10th Generation",
# )

imagine tring to get a 2d representtation of the fitness landscape?